In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import random
import numpy as np
import math
import warnings
import missingno as msno
from pprint import pprint
from chinese_calendar import is_workday, is_holiday
from pathlib import Path
from sklearn.preprocessing import PowerTransformer
import scipy.stats as st
from collections import defaultdict

# 固定随机种子，稳定模型效果
random.seed(2021) 
np.random.seed(2021)
%matplotlib inline
warnings.filterwarnings("ignore")
plt.rc("font",family="SimHei",size="10")
plt.rcParams['axes.unicode_minus']=False
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('max_colwidth',200)
pd.set_option('expand_frame_repr', False)

In [2]:
root = Path(os.getcwd()).resolve().parent / "data"
q2_df = root / "附件5：门店交易验证数据.txt"
train = root / "附件2：估价验证数据.txt"
i2n = ["carid", "pushDate", "pushPrice", "updatePriceTimeJson"]

index2name = ["carid", "展销时间", "品牌 id", "车系 id", "车型 id", "里程", "车辆颜色", "车辆所在城市 id", "国标码",
              "过户次数", "载客人数", "注册日期", "上牌日期", "国别", "厂商类型", "年款", "排量", "变速箱", "燃油类型",
              "新车价"]

# category存放的是类别特征的名称
category = ["品牌 id", "车系 id", "车型 id", "车辆颜色", "车辆所在城市 id", "厂商类型", "燃油类型"] # 类别类型特征

# numerical存放的是数值特征的名称
numerical = ["Feature_5", "年款", "排量", "变速箱", "Feature_2", "Feature_13", "载客人数", "新车价", "里程", "target"] #  "厂商类型"实数类型特征, "Feature_12", "Feature_8"

# cross_num存放的是要进行特征交叉的特征的名称
cross_num = ["新车价", "Feature_2", "Feature_12_1"]

# 匿名特征为Feature_i，此处将15个匿名特征加入到index2name中
for _ in range(1, 16):
    index2name.append("Feature_"+str(_))
# index2name.append("target")
def __i2n(df):
    df = df.rename(columns={index: _ for index, _ in enumerate(index2name)})
    return df

In [3]:
q2 = pd.read_csv(q2_df, sep='\t', header=None)
q2.rename(columns={index: _ for index, _ in enumerate(i2n)}, inplace=True)

In [4]:
train_df = pd.read_csv(train, sep='\t', header=None)
train_df = __i2n(train_df)

In [5]:
train_df.head()

,carid,展销时间,品牌 id,车系 id,车型 id,里程,车辆颜色,车辆所在城市 id,国标码,过户次数,载客人数,注册日期,上牌日期,国别,厂商类型,年款,排量,变速箱,燃油类型,新车价,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15
0,3,2021-09-26,3,3,3,6.64,2,3,1,0,7,2018-03-01,2018-08-20,779416.0,2.0,2018.0,2.0,3,1,25.98,1.0,3,2,3.0,3,1,NaN,2.0,5.0,2.0,1+2,4878*1925*1734,201710.0,1,NaN
1,4,2021-08-14,4,4,4,8.04,3,1,2,2,4,2012-11-01,2013-04-20,779411.0,3.0,2011.0,1.6,3,1,26.90,1.0,4,2,4.0,4,3,2018-06-14,1.0,3.0,2.0,1+2,3723*1683*1407,201010.0,2,NaN
2,8,2021-10-09,8,8,8,10.19,5,1,2,0,5,2012-08-01,2012-09-12,779412.0,2.0,NaN,1.6,6,1,7.58,1.0,2,2,NaN,8,2,NaN,1.0,4.0,3.0,NaN,4415*1674*1415,201003.0,1,NaN
3,9,2021-09-30,9,9,9,2.27,2,2,4,0,5,2019-12-01,2020-05-19,779413.0,1.0,2019.0,1.5,7,1,8.20,1.0,5,2,9.0,9,1,NaN,2.0,5.0,NaN,1+2,4649*1830*1705,201907.0,2,NaN
4,11,2021-08-09,8,11,11,7.03,2,3,1,0,5,2018-11-01,2019-03-08,779412.0,2.0,2019.0,2.0,7,1,21.79,1.0,6,2,11.0,10,2,NaN,2.0,4.0,NaN,1+2,4933*1836*1469,201810.0,1,NaN


In [6]:
result = pd.merge(q2, train_df, left_on="carid", right_on="carid", how="left")

In [8]:
result["展销时间"] = pd.to_datetime(result["展销时间"])
result["注册日期"] = pd.to_datetime(result["注册日期"])
result["上牌日期"] = pd.to_datetime(result["上牌日期"])
result["pushDate"] = pd.to_datetime(result["pushDate"])
# result["pullDate"] = pd.to_datetime(result["pullDate"])
# result["withdrawDate"] = pd.to_datetime(result["withdrawDate"])

In [45]:
result["成交周期"] = (result["withdrawDate"] - result["pushDate"]).map(lambda x: x.days)

In [11]:
import json
result["updatePriceTimeJson_v2"] = result[["pushDate", "pushPrice", "updatePriceTimeJson"]].apply(lambda x: {str(x["pushDate"]): x["pushPrice"]} if len(x["updatePriceTimeJson"]) == 2 else dict({str(x["pushDate"]): x["pushPrice"]}, **json.loads(x["updatePriceTimeJson"])), axis=1)
result["target"] = result["updatePriceTimeJson_v2"].map(lambda x: float(list(x.values())[-1]))
result["fday"] = result[["pushDate", "updatePriceTimeJson_v2"]].apply(lambda x: (pd.to_datetime(list(x["updatePriceTimeJson_v2"].keys())[-1]) - x["pushDate"]).days, axis=1)
del result["updatePriceTimeJson_v2"]

In [12]:
result.head()

,carid,pushDate,pushPrice,updatePriceTimeJson,展销时间,品牌 id,车系 id,车型 id,里程,车辆颜色,车辆所在城市 id,国标码,过户次数,载客人数,注册日期,上牌日期,国别,厂商类型,年款,排量,变速箱,燃油类型,新车价,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,target,fday
0,4449,2021-09-12,12.58,"{""2021-09-17"": ""12.45""}",2021-09-17,38,581,1825,5.03,1,14,1,1,7,2018-03-01,2018-04-18,779421.0,2.0,2018.0,1.8,7,1,22.98,1.0,3,2,NaN,38,1,2019-10-29,2.0,5.0,2.0,1+2,4698*1883*1676,201710.0,2,NaN,12.45,5
1,1184,2021-07-14,7.80,"{""2021-08-31"": ""7.4""}",2021-08-31,6,190,562,6.56,1,2,1,0,5,2017-09-01,2018-03-02,779413.0,1.0,2016.0,1.8,3,1,14.88,NaN,5,2,146.0,6,1,NaN,2.0,5.0,2.0,1+2,4519*1831*1694,201603.0,1,NaN,7.40,48
2,1435,2021-08-30,13.30,{},2021-08-30,19,26,1223,1.05,4,1,4,0,5,2019-12-01,2020-04-10,779413.0,1.0,2019.0,2.0,14,1,15.98,NaN,9,2,21.0,9,5,NaN,2.0,4.0,NaN,1,5410*1934*1886,201909.0,1,2035-04-10,13.30,0
3,70693,2021-08-07,5.30,{},2021-08-07,37,84,469,2.47,1,1,1,2,5,2019-11-01,2020-04-08,NaN,1.0,2019.0,1.5,10,1,9.79,1.0,3,2,67.0,11,1,2020-11-16,NaN,NaN,NaN,1,4750*1910*1710,201907.0,1,2035-04-08,5.30,0
4,749,2021-07-22,7.80,"{""2021-08-15"": ""6.9"", ""2021-08-29"": ""6.39""}",2021-08-29,65,330,693,2.87,3,4,1,1,5,2018-11-01,2019-06-25,779413.0,1.0,2019.0,1.5,11,1,11.98,1.0,3,2,257.0,74,1,2021-07-15,2.0,5.0,NaN,1+2,4685*1865*1645,201810.0,1,NaN,6.39,38


In [13]:
result.to_csv(root / "Q3_processed.csv", index=None)